In [ ]:
import pandas as pd

# Ensure the file path is correct or provide an alternative path to a valid CSV file
df = pd.read_csv('/ev_workspace/data/sap.csv')


In [ ]:
# Display the first few rows of the dataframe
print(df.head())

# Display information about the dataframe
print(df.info())

# Display statistical summary of the dataframe
print(df.describe())

In [ ]:
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

df['power'] = df['Energy'] / df['Charge.Duration'] * 60
df['charge_end'] = df['Start'] + pd.to_timedelta(df['Charge.Duration'], unit='min')
print(df.head())

In [ ]:
# Convert the 'Start' and 'End' columns to datetime
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Define the start and end times
start_time = pd.Timestamp('2017-06-01 09:00')
end_time = df['End'].max()

# Create a date range with 15-minute intervals
date_range = pd.date_range(start=start_time, end=end_time, freq='15min')

# Create a new DataFrame with the date range
new_df = pd.DataFrame(date_range, columns=['start'])
new_df['end'] = new_df['start'] + pd.Timedelta(minutes=15)

print(new_df.head())



In [ ]:
# Initialize a new column in new_df to store the summed power values
new_df['power_0'] = 0.0

# Iterate over each row in new_df
for i, row in new_df.iterrows():
    # Define the current start and end times
    current_start = row['start']
    current_end = row['end']
    
    # Filter the rows in df where the current interval overlaps with Start and charge_end
    mask = (df['Start'] < current_end) & (df['charge_end'] > current_start)
    
    # Sum the power values for the filtered rows
    summed_power = df.loc[mask, 'power'].sum()
    
    # Assign the summed power value to the new column in new_df
    new_df.at[i, 'power_0'] = summed_power

print(new_df.head())

In [ ]:
for i in range(4):
    new_df[f'power_{i}'] = new_df['power_0'].shift(-i)

print(new_df.head())

print(new_df.shape)

# new_df = new_df.iloc[:50000]

print(new_df.shape)

In [7]:

import numpy as np

X_time = pd.DataFrame()
X_time['hour_sin'] = np.sin(2 * np.pi * pd.to_datetime(new_df['start']).dt.hour / 24)
X_time['month_sin'] = np.sin(2 * np.pi * pd.to_datetime(new_df['start']).dt.month / 12)
X_time['weekday_sin'] = np.sin(2 * np.pi * pd.to_datetime(new_df['start']).dt.weekday / 7)
X_time['weekofyear_sin'] = np.sin(2 * np.pi * pd.to_datetime(new_df['start']).dt.isocalendar().week / 52)
X_time['quarter_sin'] = np.sin(2 * np.pi * pd.to_datetime(new_df['start']).dt.quarter / 4)
X_time['is_weekend'] = (pd.to_datetime(new_df['start']).dt.weekday >= 5).astype(int)

X_time['lag_1'] = new_df['power_0'].shift(1)  # Viivästetty arvo 1 aika-askeleen päähän
X_time['lag_2'] = new_df['power_0'].shift(2)  # Viivästetty arvo 2 aika-askeleen päähän
X_time['lag_3'] = new_df['power_0'].shift(3)  # Viivästetty arvo 2 aika-askeleen päähän
X_time['lag_4'] = new_df['power_0'].shift(4)  # Viivästetty arvo 2 aika-askeleen päähän
X_time['lag_8'] = new_df['power_0'].shift(8)  # Viivästetty arvo 7 aika-askeleen päähän (esim. viikon takainen arvo)

# 2. Liikkuvat keskiarvot ("rolling averages")
X_time['rolling_mean_3'] = new_df['power_0'].rolling(window=3).mean()  # 3:n edellisen havaintojakson keskiarvo
X_time['rolling_mean_7'] = new_df['power_0'].rolling(window=7).mean()  # 7:n edellisen havaintojakson keskiarvo

# 3. Liikkuvat keskihajonnat ("rolling standard deviations")
X_time['rolling_std_3'] = new_df['power_0'].rolling(window=3).std()  # 3:n edellisen havaintojakson keskihajonta
X_time['rolling_std_7'] = new_df['power_0'].rolling(window=7).std()  # 7:n edellisen havaintojakson keskihajonta

# 4. Kuukausi- ja viikkosummat
X_time['rolling_sum_14'] = new_df['power_0'].rolling(window=14).sum()  # Kuukauden summat (olettaen päivittäiset tiedot)
X_time['rolling_sum_7'] = new_df['power_0'].rolling(window=7).sum()    # Viikon summat
X_time['rolling_sum_3'] = new_df['power_0'].rolling(window=3).sum()    # Viikon summat

# 5. Edellisen päivän korkeimmat ja alhaisimmat arvot
X_time['lag_max_3'] = new_df['power_0'].rolling(window=3).max()  # Edellisen päivän maksimiarvo
X_time['lag_min_3'] = new_df['power_0'].rolling(window=3).min()  # Edellisen päivän minimiarvo
X_time['lag_max_8'] = new_df['power_0'].rolling(window=8).max()  # Edellisen päivän maksimiarvo
X_time['lag_min_8'] = new_df['power_0'].rolling(window=8).min()  # Edellisen päivän minimiarvo

# Liikkuvat mediaanit ("rolling medians")
X_time['rolling_median_3'] = new_df['power_0'].rolling(window=3).median()  # 3:n edellisen havaintojakson mediaani
X_time['rolling_median_7'] = new_df['power_0'].rolling(window=7).median()  # 7:n edellisen havaintojakson mediaani
X_time['rolling_median_14'] = new_df['power_0'].rolling(window=14).median()  # 30:n edellisen havaintojakson mediaani

X_time = X_time.dropna()
X_time = X_time.values


In [8]:
# Määritellään suhteet koulutus-, validointi- ja testidatalle
training_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Määritellään jakopisteet
training_size = int(len(new_df) * training_ratio)
validation_size = int(len(new_df) * validation_ratio)
test_size = len(new_df) - training_size - validation_size
# Jaetaan data kolmeen osaan
training_data = new_df[:training_size]
validation_data = new_df[training_size:training_size + validation_size]
test_data = new_df[training_size + validation_size:]

# Jaetaan myös X_time kolmeen osaan
training_X_time = X_time[:training_size]
validation_X_time = X_time[training_size:training_size + validation_size]
test_X_time = X_time[training_size + validation_size:]


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
power_scaler = StandardScaler()

# Fit the scaler on 'power_0' of the training data
power_scaler.fit(training_data['power_0'].values.reshape(-1, 1))

# Scale 'power_0' and target columns in the training data
training_data_scaled = training_data.copy()
training_data_scaled['power_0'] = power_scaler.transform(training_data[['power_0']])

for i in range(4):
    training_data_scaled[f'power_{i}'] = power_scaler.transform(training_data[[f'power_{i}']])

# Scale 'power_0' and target columns in the validation data
validation_data_scaled = validation_data.copy()
validation_data_scaled['power_0'] = power_scaler.transform(validation_data[['power_0']])

for i in range(4):
    validation_data_scaled[f'power_{i}'] = power_scaler.transform(validation_data[[f'power_{i}']])

# Scale 'power_0' and target columns in the test data
test_data_scaled = test_data.copy()
test_data_scaled['power_0'] = power_scaler.transform(test_data[['power_0']])

for i in range(4):
    test_data_scaled[f'power_{i}'] = power_scaler.transform(test_data[[f'power_{i}']])


In [10]:
times_scaler = StandardScaler()

# Fit the scaler on 'power_0' of the training data
times_scaler.fit(training_X_time)

# Scale 'power_0' and target columns in the training data
training_X_time_scaled = training_X_time.copy()
training_X_time_scaled = times_scaler.transform(training_X_time)

# Scale 'power_0' and target columns in the validation data
validation_X_time_scaled = validation_X_time.copy()
validation_X_time_scaled = times_scaler.transform(validation_X_time)

# Scale 'power_0' and target columns in the test data
test_X_time_scaled = test_X_time.copy()
test_X_time_scaled = times_scaler.transform(test_X_time)


In [ ]:
future_window = 4
import optuna
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Input, concatenate, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D, Dropout, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import time
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sklearn.linear_model import LinearRegression

def combined_window_dataset(data, X_time, past_window, future_window, batch_size):
    data_len = len(data)
    
    def generator():
        i = past_window
        while True:
            X_batch = []
            X_time_batch = []
            y_batch = []
            for _ in range(batch_size):
                if i + past_window + future_window >= data_len:
                    i = past_window  # Palataan alkuun, kun data loppuu

                # Otetaan menneet arvot ikkunana (X_batch)
                X = data['power_0'].values[i:i + past_window]
                X_batch.append(X)

                # Otetaan ajalliset ominaisuudet (X_time_batch)
                X_time_batch.append(X_time[i])

                # Otetaan tulevaisuuden arvot, joita halutaan ennustaa (y_batch)
                y = data['power_0'].values[i + past_window: i + past_window + future_window]
                y_batch.append(y)

                i += 1

            # Muunnetaan numpy-arrayksi ja muotoillaan se oikeaan muotoon
            X_batch = np.array(X_batch).reshape(batch_size, past_window, 1)  # Muodoksi (batch_size, past_window, 1)
            X_time_batch = np.array(X_time_batch).reshape(batch_size, -1)  # Muodoksi (batch_size, feature_count)
            y_batch = np.array(y_batch).reshape(batch_size, future_window)  # Muodoksi (batch_size, future_window)

            yield (X_batch, X_time_batch), y_batch

    output_signature = (
        (tf.TensorSpec(shape=(batch_size, past_window, 1), dtype=tf.float32),
         tf.TensorSpec(shape=(batch_size, X_time.shape[1]), dtype=tf.float32)),
        tf.TensorSpec(shape=(batch_size, future_window), dtype=tf.float32)
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature)


# Generaattori koulutusta varten




# Optuna - CNN mallin optimointi

def create_model(trial, past_window, future_window):
    # Optuna hakee hyperparametrit
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)    
    stride = trial.suggest_int("stride", 2, 10)
    n_dense_units_combined_1 = trial.suggest_int("n_dense_units_combined_1", 32, 256, log=True)
    dropout_rate_1 = trial.suggest_float("dropout_rate_1", 0.1, 0.8)

    # Input layer
    input_layer = Input(shape=(past_window, 1))

    # Ensimmäinen pää (kuten alkuperäinen malli)
    n_filters_1 = trial.suggest_int("n_filters_1", 4, 128, log=True)
    kernel_size_1 = trial.suggest_int("kernel_size_1", 2, 10)
    pooling_1 = trial.suggest_categorical("pooling_1", ['max', 'average', 'global_max', 'global_average'])
    
    x1 = Conv1D(filters=n_filters_1, kernel_size=kernel_size_1, activation='relu')(input_layer)
    if pooling_1 == 'max':
        x1 = tf.keras.layers.MaxPooling1D(pool_size=2)(x1)
    elif pooling_1 == 'global_max':
        x1 = GlobalMaxPooling1D()(x1)
    elif pooling_1 == 'global_average':
        x1 = GlobalAveragePooling1D()(x1)
    else:
        x1 = tf.keras.layers.AveragePooling1D(pool_size=2)(x1)

    x1 = Flatten()(x1)

    # Toinen pää (optimoitu stride)
    n_filters_2 = trial.suggest_int("n_filters_2", 4, 128, log=True)
    kernel_size_2 = trial.suggest_int("kernel_size_2", 2, 10)
    pooling_2 = trial.suggest_categorical("pooling_2", ['max', 'average', 'global_max', 'global_average'])
    
    x2 = Conv1D(filters=n_filters_2, kernel_size=kernel_size_2, strides=stride, activation='relu')(input_layer)
    if pooling_2 == 'max':
        x2 = tf.keras.layers.MaxPooling1D(pool_size=2)(x2)
    elif pooling_2 == 'global_max':
        x2 = GlobalMaxPooling1D()(x2)
    elif pooling_2 == 'global_average':
        x2 = GlobalAveragePooling1D()(x2)
    else:
        x2 = tf.keras.layers.AveragePooling1D(pool_size=2)(x2)
        
    x2 = Flatten()(x2)

    # Kolmas pää (kaksi 1D CNN-kerrosta allekkain)
    n_filters_3 = trial.suggest_int("n_filters_3", 4, 128, log=True)   
    kernel_size_3 = trial.suggest_int("kernel_size_3", 2, 10)
    pooling_3 = trial.suggest_categorical("pooling_3", ['max', 'average', 'global_max', 'global_average'])
    
    x3 = Conv1D(filters=n_filters_3, kernel_size=kernel_size_3, activation='relu')(input_layer)    
    if pooling_3 == 'max':
        x3 = tf.keras.layers.MaxPooling1D(pool_size=2)(x3)
    elif pooling_3 == 'global_max':
        x3 = GlobalMaxPooling1D()(x3)
    elif pooling_3 == 'global_average':
        x3 = GlobalAveragePooling1D()(x3)
    else:
        x3 = tf.keras.layers.AveragePooling1D(pool_size=2)(x3)

    x3 = Flatten()(x3)

    input_time = Input(shape=(X_time.shape[1],), name="time_input")

    # Yhdistetään kaikki päät
    combined = concatenate([x1, x2, x3, input_time])

    # Syvä kerros yhdistetylle datalle
    combined = Dense(n_dense_units_combined_1, activation='relu')(combined)
    combined = Dropout(dropout_rate_1)(combined)
    combined = Dense(n_dense_units_combined_1, activation='relu')(combined)
    combined = Dropout(dropout_rate_1)(combined)

    # Lopullinen ulostulo
    output = Dense(future_window)(combined)

    # Mallin luominen
    model = Model(inputs=[input_layer, input_time], outputs=output)

    # Optimizerin asetukset
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model


def objective(trial):
    try:
        start_time = time.time()  # Start timing

        # Hyperparameters
        past_window = trial.suggest_int("past_window", 100, 3000, log=True)
        batch_size = trial.suggest_int("batch_size", 32, 128, log=True)
        future_window = 4  # As defined earlier

        # Prepare datasets using scaled data
        train_dataset = combined_window_dataset(
            training_data_scaled, training_X_time_scaled, past_window, future_window, batch_size
        ).prefetch(tf.data.experimental.AUTOTUNE)
        validation_dataset = combined_window_dataset(
            validation_data_scaled, validation_X_time_scaled, past_window, future_window, batch_size
        ).prefetch(tf.data.experimental.AUTOTUNE)

        # Model setup
        model = create_model(trial, past_window=past_window, future_window=future_window)
        steps_per_epoch_train = len(training_data_scaled) // batch_size
        steps_per_epoch_val = len(validation_data_scaled) // batch_size

        # Training parameters
        EPOCHS = 20
        val_losses = []
        MAX_TIME = 300  # Maximum time per trial in seconds

        # Training loop with pruning and time checking
        for epoch in range(EPOCHS):
            # Check elapsed time
            time_elapsed = time.time() - start_time
            if time_elapsed > MAX_TIME:
                print(f"Trial {trial.number} stopped at epoch {epoch} due to time limit")
                raise optuna.TrialPruned()  # Prune if time limit exceeded

            # Train for one epoch
            history = model.fit(
                train_dataset,
                steps_per_epoch=steps_per_epoch_train,
                validation_data=validation_dataset,
                validation_steps=steps_per_epoch_val,
                epochs=1,
                verbose=2
            )

            # Append validation loss to list
            val_losses.append(history.history['val_loss'][-1])

            running_mean = np.mean(val_losses[-5:])                

            print(f'Running mean of validation loss: {running_mean}')

            trial.report(running_mean, step=epoch)

            if trial.should_prune():
                print(f"Trial {trial.number} pruned at epoch {epoch}")
                raise optuna.TrialPruned()

        running_mean = np.mean(val_losses[-5:])

        # Clean up
        del model, train_dataset, validation_dataset, history
        tf.keras.backend.clear_session()

        print(f'Time taken for this trial: {time.time() - start_time}')

        return running_mean

    except optuna.TrialPruned:
        # Käsitellään prunatussa tilassa, ilman että merkitään virheeksi
        print(f"Trial {trial.number} pruned.")
        raise  # Pruned trials are re-raised to notify Optuna

    except Exception as e:
        # Käsitellään muut poikkeukset varsinaisina virheinä
        print(f"Trial {trial.number} failed with exception: {e}")
        trial.set_user_attr("failed", True)
        raise


pruner = MedianPruner(
    n_startup_trials=10,      # Pruneri odottaa vähintään 10 kokeilua ennen kuin alkaa pruneamaan
    n_warmup_steps=5,         # Alkuun vähintään 5 epochia, jotta mallin trendi saadaan näkyviin
    interval_steps=1          # Prunaus tapahtuu joka toisen epochin jälkeen
)

sampler = TPESampler(
    n_startup_trials=10,      # Aluksi käytetään satunnaisotantaa 10 kokeilun ajan
    n_ei_candidates=24,       # Joka vaiheessa valitaan 24 ehdokasta Expected Improvement -estimaatin avulla
    multivariate=True,        # Multivariaattinen versio TPE:stä, joka voi parantaa konvergenssia
    group=True                # Asettaa ehdokkaat ryhmiksi, joka voi johtaa parempiin päätöksiin
)


# Optuna tutkimuksen suoritus SQL-tietokannan kanssa
study = optuna.create_study(
    direction="minimize",  # Maksimoidaan kulmakerrointa
    sampler=sampler,  # Käytetään määriteltyä TPE-sampleria
    pruner=pruner,    # Käytetään määriteltyä median pruneria
    storage="sqlite:///optuna_study.db",  # SQLite-tietokanta
    load_if_exists=True,
    study_name='2024_10_29_slopet_tyo'  # Lataa olemassa oleva tutkimus, jos se on olemassa
)

study.optimize(objective, timeout= 0.5 * 3600)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

tf.keras.backend.clear_session()

def create_model(best_params, past_window, future_window):
    # Parametrien noutaminen suoraan sanakirjasta
    learning_rate = best_params["learning_rate"]    
    stride = best_params["stride"]
    n_dense_units_combined_1 = best_params["n_dense_units_combined_1"]
    dropout_rate_1 = best_params["dropout_rate_1"]

    # Input layer
    input_layer = Input(shape=(past_window, 1))

    # Ensimmäinen pää (kuten alkuperäinen malli)
    n_filters_1 = best_params["n_filters_1"]
    kernel_size_1 = best_params["kernel_size_1"]
    pooling_1 = best_params["pooling_1"]

    x1 = Conv1D(filters=n_filters_1, kernel_size=kernel_size_1, activation='relu')(input_layer)
    if pooling_1 == 'max':
        x1 = MaxPooling1D(pool_size=2)(x1)
    else:
        x1 = AveragePooling1D(pool_size=2)(x1)

    x1 = Conv1D(filters=n_filters_1, kernel_size=kernel_size_1, activation='relu')(x1)
    if pooling_1 == 'max':
        x1 = MaxPooling1D(pool_size=2)(x1)
    else:
        x1 = AveragePooling1D(pool_size=2)(x1)

    x1 = Flatten()(x1)

    # Toinen pää (optimoitu stride)
    n_filters_2 = best_params["n_filters_2"]
    kernel_size_2 = best_params["kernel_size_2"]
    pooling_2 = best_params["pooling_2"]

    x2 = Conv1D(filters=n_filters_2, kernel_size=kernel_size_2, strides=stride, activation='relu')(input_layer)
    if pooling_2 == 'max':
        x2 = MaxPooling1D(pool_size=2)(x2)
    else:
        x2 = AveragePooling1D(pool_size=2)(x2)

    x2 = Conv1D(filters=n_filters_2, kernel_size=kernel_size_2, strides=stride, activation='relu')(x2)
    if pooling_2 == 'max':
        x2 = MaxPooling1D(pool_size=2)(x2)
    else:
        x2 = AveragePooling1D(pool_size=2)(x2)

    x2 = Flatten()(x2)

    # Kolmas pää (kaksi 1D CNN-kerrosta allekkain)
    n_filters_3 = best_params["n_filters_3"]
    kernel_size_3 = best_params["kernel_size_3"]
    pooling_3 = best_params["pooling_3"]

    x3 = Conv1D(filters=n_filters_3, kernel_size=kernel_size_3, activation='relu')(input_layer)
    if pooling_3 == 'max':
        x3 = MaxPooling1D(pool_size=2)(x3)
    else:
        x3 = AveragePooling1D(pool_size=2)(x3)

    x3 = Conv1D(filters=n_filters_3, kernel_size=kernel_size_3, activation='relu')(x3)
    if pooling_3 == 'max':
        x3 = MaxPooling1D(pool_size=2)(x3)
    else:
        x3 = AveragePooling1D(pool_size=2)(x3)

    x3 = Flatten()(x3)

    # Ajallinen syöte
    input_time = Input(shape=(X_time.shape[1],), name="time_input")

    # Yhdistetään kaikki päät
    combined = concatenate([x1, x2, x3, input_time])

    # Syvä kerros yhdistetylle datalle
    combined = Dense(n_dense_units_combined_1, activation='relu')(combined)
    combined = Dropout(dropout_rate_1)(combined)
    combined = Dense(n_dense_units_combined_1, activation='relu')(combined)
    combined = Dropout(dropout_rate_1)(combined)

    # Lopullinen ulostulo
    output = Dense(future_window)(combined)

    # Mallin luominen
    model = Model(inputs=[input_layer, input_time], outputs=output)

    # Optimizerin asetukset
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

# Ensure that 'power_scaler' is the same scaler used during training
# If it was saved, load it; otherwise, make sure it's accessible here

# Training the best model
best_params = study.best_params
batch_size = best_params.pop("batch_size")
past_window = best_params.pop("past_window")
future_window = 4  # As defined earlier

# Create the final model with the best hyperparameters
final_model = create_model(best_params, past_window=past_window, future_window=future_window)

# Prepare datasets (ensure you're using scaled data)
train_dataset = combined_window_dataset(training_data_scaled, training_X_time_scaled, past_window, future_window, batch_size).prefetch(tf.data.experimental.AUTOTUNE)
validation_dataset = combined_window_dataset(validation_data_scaled, validation_X_time_scaled, past_window, future_window, batch_size).prefetch(tf.data.experimental.AUTOTUNE)

steps_per_epoch_train = len(training_data_scaled) // batch_size
steps_per_epoch_val = len(validation_data_scaled) // batch_size


final_model.summary()

# Define callbacks
checkpoint_callback = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, min_lr=1e-6, verbose=1)

# Train the model
final_model.fit(train_dataset,
                validation_data=validation_dataset,
                epochs=100,  # Adjust as needed
                callbacks=[checkpoint_callback, reduce_lr_callback],
                steps_per_epoch=steps_per_epoch_train,                
                validation_steps=steps_per_epoch_val,                           
                verbose=2)

# Load the best model
best_model = tf.keras.models.load_model('best_model.keras')

# Testing the model with test data
y_true_scaled = []
y_pred_scaled = []

test_dataset = combined_window_dataset(test_data_scaled, test_X_time_scaled, past_window, future_window, batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Collect predictions and true values from the test set
for (X_batch, X_time_batch), y_batch in test_dataset:
    predictions = best_model.predict([X_batch, X_time_batch], verbose=0)
    y_true_scaled.extend(y_batch.numpy())     # True values in scaled form
    y_pred_scaled.extend(predictions)         # Predictions in scaled form
    # Break after one full epoch to prevent infinite loop
    if len(y_true_scaled) >= len(test_data_scaled) - past_window - future_window:
        break

# Convert lists to numpy arrays
y_true_scaled = np.array(y_true_scaled)
y_pred_scaled = np.array(y_pred_scaled)

# Inverse-transform the scaled predictions and true values
# Reshape if necessary to ensure correct dimensions
y_true_inv = power_scaler.inverse_transform(y_true_scaled)
y_pred_inv = power_scaler.inverse_transform(y_pred_scaled)

# Calculate MAE, MAPE, and MSE for each prediction window
for i in range(future_window):

    mae = mean_absolute_error(y_true_inv[:, i], y_pred_inv[:, i])
    mape = mean_absolute_percentage_error(y_true_inv[:, i], y_pred_inv[:, i])
    mse = mean_squared_error(y_true_inv[:, i], y_pred_inv[:, i])
    print(f"Prediction Step {i+1}:")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"MSE: {mse}")
    print("------------------------------")

# Overall metrics across all prediction steps
overall_mae = mean_absolute_error(y_true_inv.flatten(), y_pred_inv.flatten())
overall_mape = mean_absolute_percentage_error(y_true_inv.flatten(), y_pred_inv.flatten())
overall_mse = mean_squared_error(y_true_inv.flatten(), y_pred_inv.flatten())
print("Overall Metrics:")
print(f"MAE: {overall_mae}")
print(f"MAPE: {overall_mape}")
print(f"MSE: {overall_mse}")

# Examine the first prediction
print("First prediction (inverse-transformed):", y_pred_inv[0])
print("Actual values (inverse-transformed):", y_true_inv[0])
